In [2]:
import os

from ultralytics import YOLO
import cv2
from matplotlib import pyplot as plt

model = YOLO("best.pt")

In [3]:
saved_frame_count = 0

def cropping_img(frame, coordinates):
    global saved_frame_count
    if len(coordinates) != 4:
        print(coordinates)
        raise ValueError("Неправильный формат coordinates.\n")
    
    x1, y1, x2, y2 = coordinates

    if x1 < 0 or y1 < 0 or x2 > frame.shape[1] or y2 > frame.shape[0]:
        raise ValueError("Неправильный формат записи координат.")

    cropped_image = frame[y1:y2, x1:x2]
    
    # Сохранение изображения
    file_name = f"../selected_videos/frames/{saved_frame_count}.jpg"
    cv2.imwrite(file_name, cropped_image)
    saved_frame_count += 1

In [4]:
def people_detect(frame):
    result = model(frame)
    
    if result and len(result) > 0:
        for detection in result[0].boxes:

            # Извлекаем информацию о координатах и классе
            x1, y1, x2, y2 = detection.xyxy[0]  # Получаем координаты ограничивающей рамки
            confidence = detection.conf[0]  # Получаем уровень уверенности
            class_id = int(detection.cls[0])  # Получаем идентификатор класса

            if class_id == 0:  # Проверяем, является ли класс человеком
                print(f"Обнаружен человек с уверенностью {confidence:.2f} на координатах ({x1:.0f}, {y1:.0f}, {x2:.0f}, {y2:.0f})")
                cropping_img(frame, (int(x1), int(y1), int(x2), int(y2)))  # Вызов функции для обрезки изображения

In [5]:
def extract_frames(video_directory, frame_interval):
    # Получаем список всех файлов в директории видео
    video_files = [f for f in os.listdir(video_directory) if f.endswith(('.mp4', '.avi', '.mov'))]

    for video_file in video_files:
        video_path = os.path.join(video_directory, video_file)
        # Загружаем видео
        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            print(f"Не удалось открыть видео: {video_path}")
            continue

        frame_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break  # Выход из цикла, если больше нет кадров

            # Сохраняем только каждый n-й кадр
            if frame_count % frame_interval == 0:
                people_detect(frame)
            
            frame_count += 1

        cap.release()
        print(f"Извлечено {frame_count} кадров из {video_file}")
        print(saved_frame_count)

video_directory = '../selected_videos/videos'
frame_interval = 20

extract_frames(video_directory, frame_interval)


0: 384x640 2 Personas, 52.8ms
Speed: 0.0ms preprocess, 52.8ms inference, 223.8ms postprocess per image at shape (1, 3, 384, 640)
Обнаружен человек с уверенностью 0.81 на координатах (257, 561, 488, 1013)
Обнаружен человек с уверенностью 0.73 на координатах (742, 415, 910, 790)

0: 384x640 2 Personas, 47.9ms
Speed: 0.0ms preprocess, 47.9ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
Обнаружен человек с уверенностью 0.83 на координатах (228, 542, 470, 998)
Обнаружен человек с уверенностью 0.83 на координатах (727, 418, 911, 820)

0: 384x640 2 Personas, 40.6ms
Speed: 0.0ms preprocess, 40.6ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)
Обнаружен человек с уверенностью 0.86 на координатах (225, 542, 456, 989)
Обнаружен человек с уверенностью 0.84 на координатах (727, 421, 903, 804)

0: 384x640 3 Personas, 41.1ms
Speed: 0.0ms preprocess, 41.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Обнаружен человек с уверенностью 0.85 на 